In [25]:
import pandas  as pd
import pymongo as db
import config
from bson.objectid import ObjectId
from typing import Callable

In [2]:
client = db.MongoClient(config.MONGO_URI)

try:
    database = client.get_database("saturnDB")
    collection = database.get_collection("metrics")    
    query = { "organization_id": ObjectId("6745d7873f4e39e161319575"),
              "year": 2023,
              "month": 1 }
    metrics = collection.find_one(query)
    client.close()
except Exception as e:
    raise Exception("Unable to retrieve the document due to the following error: ", e)

if (not metrics):
    raise Exception("Collection not found.")
else:
    try:
        df = pd.json_normalize(metrics)
    except ValueError as e:
        raise Exception("Unable to load data from the collection: ", e)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 99 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   _id                                    1 non-null      object
 1   organization_id                        1 non-null      object
 2   year                                   1 non-null      int64 
 3   month                                  1 non-null      int64 
 4   partos_vaginais                        1 non-null      int64 
 5   partos_cesareos                        1 non-null      int64 
 6   saidas_clinicas_anterior               1 non-null      int64 
 7   saidas_cirurgicas_anterior             1 non-null      int64 
 8   reinternacoes_clinicas                 1 non-null      int64 
 9   reinternacoes_cirurgicas               1 non-null      int64 
 10  pacientes_dia                          1 non-null      int64 
 11  pcr_eventos            

In [ ]:
a = [,df.partos_cesareos]
len(a)

2

In [20]:
def minha_funcao(**kwargs):
    #for chave, valor in kwargs.items():
    #    print(f"{chave}: {valor}")
    print(len(kwargs))
    print(kwargs['nome'])


In [21]:
# Chamando a função com argumentos nomeados
minha_funcao(nome="Rodrigo", idade=32, cidade="São Paulo")

3
Rodrigo


In [57]:
def valida_metrica(calculo: Callable, **kwargs):        
    if (len(kwargs) != 2):
        raise Exception(f"Necessário dois valores (numerador e denominador). Foram informados {len(kwargs)}.")
    else:
        return calculo(kwargs.get('numerador'),kwargs.get('denominador')) if kwargs.get('denominador')[0] != 0 else None
    
def kpi_taxa(numerador,denominador):
    return (numerador/denominador)[0] *100

def kpi_tempo_medio(numerador,denominador):
    return (numerador/denominador)[0]

def kpi_densidade(numerador,denominador):
    return (numerador/denominador)[0] *1000

In [ ]:
a = ['cli_total_pacientes_risco_profilaxia_TEV','cli_total_pacientes_risco', 'cir_orto_total_pacientes_risco_profilaxia_TEV', 'cir_orto_total_pacientes_risco', 'cir_n_orto_total_pacientes_risco_profilaxia_TEV', 'cir_n_orto_total_pacientes_risco']


In [75]:
def kpi_taxa(numerador,denominador):
    return (numerador/denominador)[0] *100

def kpi1(**kwargs):
    # 1. Proporção de partos vaginais 

    # Validar se kwargs está vazio
    if not kwargs:
        raise ValueError("Os argumentos kwargs não podem estar vazios.")

    # Validar se existem os argumentos necessários
    if 'total_partos_vaginais' not in kwargs or 'total_partos_cesareos' not in kwargs:
        raise KeyError("Os valores de 'total_partos_vaginais' e 'total_partos_cesareos' são obrigatórios.")

    # Validar se os valores não são nulos
    if kwargs.get('total_partos_vaginais') is None or kwargs.get('total_partos_cesareos') is None:
        raise ValueError("Os valores de 'total_partos_vaginais' e 'total_partos_cesareos' não podem ser nulos.")
    
    # Validar se o número de argumentos é correto
    if len(kwargs) != 2:
        raise Exception(f"KPI1 requer 2 elementos. Foram passados {len(kwargs)}.")
    
    # Calcular o KPI
    total_partos = kwargs['total_partos_vaginais'] + kwargs['total_partos_cesareos']
    
    return kpi_taxa( numerador = kwargs['total_partos_vaginais'],
                            denominador = total_partos )

In [76]:
kpi1(total_partos_vaginais=df.partos_vaginais, total_partos_cesareos=df.partos_cesareos)

53.36787564766839

In [33]:
(df.partos_vaginais + df.partos_cesareos)[0] != 0

True

In [ ]:
df.at['']

In [58]:
valida_metrica(kpi_taxa,numerador=df.partos_vaginais,denominador=(df.partos_vaginais + df.partos_cesareos))

53.36787564766839

In [ ]:
# 1. Proporção de partos vaginais   
kpi1 = ( df.partos_vaginais / (df.partos_vaginais + df.partos_cesareos) )[0] *100

# 2. Proporção de reinternações em até 30 dias da saída hospitalar
kpi2_cli = ( df.reinternacoes_clinicas / df.saidas_clinicas_anterior )[0] *100
kpi2_cir = ( df.reinternacoes_cirurgicas / df.saidas_cirurgicas_anterior )[0] *100
kpi2 = ((df.reinternacoes_clinicas+df.reinternacoes_cirurgicas) / (df.saidas_clinicas_anterior+df.saidas_cirurgicas_anterior))[0]* 100

# 3. Taxa de parada cardiorrespiratória em unidade de internação
kpi3 = ( df.pcr_eventos / df.pacientes_dia )[0] *1000

# 4. Taxa de mortalidade institucional
kpi4_cli_neo_precoce = ( df.cli_neo_precoce_obitos / df.cli_neo_precoce_saidas )[0] *100
kpi4_cli_neo_tardio  = ( df.cli_neo_tardio_obitos  / df.cli_neo_tardio_saidas  )[0] *100
kpi4_cli_pedi        = ( df.cli_pedi_obitos        / df.cli_pedi_saidas        )[0] *100
kpi4_cli_adulto      = ( df.cli_ad_obitos          / df.cli_ad_saidas          )[0] *100
kpi4_cli_idoso       = ( df.cli_idoso_obitos       / df.cli_idoso_saidas       )[0] *100

kpi4_cir_neo_precoce = ( df.cir_neo_precoce_obitos / df.cir_neo_precoce_saidas )[0] *100
kpi4_cir_neo_tardio  = ( df.cir_neo_tardio_obitos  / df.cir_neo_tardio_saidas  )[0] *100
kpi4_cir_pedi        = ( df.cir_pedi_obitos        / df.cir_pedi_saidas        )[0] *100
kpi4_cir_adulto      = ( df.cir_ad_obitos          / df.cir_ad_saidas          )[0] *100
kpi4_cir_idoso       = ( df.cir_idoso_obitos       / df.cir_idoso_saidas       )[0] *100

kpi4_cli = (
    ( df.cli_neo_precoce_obitos+
        df.cli_neo_tardio_obitos+
        df.cli_pedi_obitos+
        df.cli_ad_obitos+
        df.cli_idoso_obitos ) / 
    ( df.cli_neo_precoce_saidas+
        df.cli_neo_tardio_saidas+
        df.cli_pedi_saidas+
        df.cli_ad_saidas+
        df.cli_idoso_saidas )
)[0] *100

kpi4_cir = (
    ( df.cir_neo_precoce_obitos+
        df.cir_neo_tardio_obitos+
        df.cir_pedi_obitos+
        df.cir_ad_obitos+
        df.cir_idoso_obitos ) / 
    ( df.cir_neo_precoce_saidas+
        df.cir_neo_tardio_saidas+
        df.cir_pedi_saidas+
        df.cir_ad_saidas+
        df.cir_idoso_saidas )
)[0] *100

kpi4_neo_precoce = (
    ( df.cli_neo_precoce_obitos+df.cir_neo_precoce_obitos ) / 
    ( df.cli_neo_precoce_saidas+df.cir_neo_precoce_saidas )
)[0] *100

kpi4_neo_tardio = (
    ( df.cli_neo_tardio_obitos+df.cir_neo_tardio_obitos ) / 
    ( df.cli_neo_tardio_saidas+df.cir_neo_tardio_saidas )
)[0] *100

kpi4_pedi = (
    ( df.cli_pedi_obitos+df.cir_pedi_obitos ) / 
    ( df.cli_pedi_saidas+df.cir_pedi_saidas )
)[0] *100

kpi4_ad = (
    ( df.cli_ad_obitos+df.cir_ad_obitos ) / 
    ( df.cli_ad_saidas+df.cir_ad_saidas )
)[0] *100

kpi4_idoso = (
    ( df.cli_idoso_obitos+df.cir_idoso_obitos ) / 
    ( df.cli_idoso_saidas+df.cir_idoso_saidas )
)[0] *100

kpi4_cir = (
    ( df.cir_neo_precoce_obitos+
        df.cir_neo_tardio_obitos+
        df.cir_pedi_obitos+
        df.cir_ad_obitos+
        df.cir_idoso_obitos ) / 
    ( df.cir_neo_precoce_saidas+
        df.cir_neo_tardio_saidas+
        df.cir_pedi_saidas+
        df.cir_ad_saidas+
        df.cir_idoso_saidas )
)[0] *100    

kpi4 = ((
    df.cli_neo_precoce_obitos+
    df.cli_neo_tardio_obitos+
    df.cli_pedi_obitos+
    df.cli_ad_obitos+
    df.cli_idoso_obitos+
    df.cir_neo_precoce_obitos+
    df.cir_neo_tardio_obitos+
    df.cir_pedi_obitos+
    df.cir_ad_obitos+
    df.cir_idoso_obitos
) / (
    df.cli_neo_precoce_saidas+
    df.cli_neo_tardio_saidas+
    df.cli_pedi_saidas+
    df.cli_ad_saidas+
    df.cli_idoso_saidas+
    df.cir_neo_precoce_saidas+
    df.cir_neo_tardio_saidas+
    df.cir_pedi_saidas+
    df.cir_ad_saidas+
    df.cir_idoso_saidas
))[0] *100

# 5. Tempo médio de internação
kpi5_cli_pedi   = ( df.cli_pedi_pacientes_dia  / df.cli_pedi_saidas)[0]
kpi5_cli_adulto = ( df.cli_ad_pacientes_dia    / df.cli_ad_saidas)[0]
kpi5_cli_idoso  = ( df.cli_idoso_pacientes_dia / df.cli_idoso_saidas)[0]
kpi5_cir_pedi   = ( df.cir_pedi_pacientes_dia  / df.cir_pedi_saidas)[0]
kpi5_cir_adulto = ( df.cir_ad_pacientes_dia    / df.cir_ad_saidas)[0]
kpi5_cir_idoso  = ( df.cir_idoso_pacientes_dia / df.cir_idoso_saidas)[0]
kpi5_cli = ( ( df.cli_pedi_pacientes_dia+df.cli_ad_pacientes_dia+df.cli_idoso_pacientes_dia ) / 
                ( df.cli_pedi_saidas+df.cli_ad_saidas+df.cli_idoso_saidas ) )[0]
kpi5_cir = ( ( df.cir_pedi_pacientes_dia+df.cir_ad_pacientes_dia+df.cir_idoso_pacientes_dia ) / 
                ( df.cir_pedi_saidas+df.cir_ad_saidas+df.cir_idoso_saidas ) )[0]
kpi5_pedi = ( ( df.cli_pedi_pacientes_dia+df.cir_pedi_pacientes_dia ) / 
                ( df.cli_pedi_saidas+df.cir_pedi_saidas ) )[0] *100
kpi5_ad = ( ( df.cli_ad_pacientes_dia+df.cir_ad_pacientes_dia ) / 
            ( df.cli_ad_saidas+df.cir_ad_saidas ) )[0]
kpi5_idoso = ( ( df.cli_idoso_pacientes_dia+df.cir_idoso_pacientes_dia ) / 
                ( df.cli_idoso_saidas+df.cir_idoso_saidas ) )[0]
kpi5 =  (( df.cli_pedi_pacientes_dia+df.cli_ad_pacientes_dia+df.cli_idoso_pacientes_dia+
            df.cir_pedi_pacientes_dia+df.cir_ad_pacientes_dia+df.cir_idoso_pacientes_dia ) / (
            df.cli_pedi_saidas+df.cli_ad_saidas+df.cli_idoso_saidas+
            df.cir_pedi_saidas+df.cir_ad_saidas+df.cir_idoso_saidas ))[0]

# 6. Tempo médio de permanência na emergência
kpi6 = ( df.total_tempo_permanencia_emergencia_hr / df.total_pacientes_emergencia )[0]

# 7. Tempo médio de espera na emergência para primeiro atendimento
kpi7_nvl2 = ( df.tempo_total_emergencia_nivel2_min / df.pacientes_emergencia_nivel2 )[0]
kpi7_nvl3 = ( df.tempo_total_emergencia_nivel3_min / df.pacientes_emergencia_nivel3 )[0]
kpi7 = ( ( df.tempo_total_emergencia_nivel2_min+df.tempo_total_emergencia_nivel3_min ) /
            ( df.pacientes_emergencia_nivel2+df.pacientes_emergencia_nivel3 ) )[0]

# 8. Taxa de início de antibiótico intravenoso profilático
kpi8 = ( df.cirurgias_com_antibiotico / df.total_cirurgias_limpas )[0] *100

# 9. Taxa de infecção de sítio cirúrgico em cirurgia limpa
kpi9 = ( df.total_infeccoes / df.total_cirurgias_limpas )[0] *100

# 10. Densidade de incidência de infecção primária de corrente sanguínea (IPCS)
# em pacientes em uso de cateter venoso central (CVC)
kpi10_neo = ( ( df.unid_int_neo_precoce_infec + df.unid_int_neo_tardio_infec ) / 
                ( df.unid_int_neo_precoce_cvc_dia + df.unid_int_neo_tardio_cvc_dia ) )[0] *1000
kpi10_pedi = ( df.unid_int_pedi_infec / df.unid_int_pedi_cvc_dia )[0] *1000
kpi10_ad = ( ( df.unid_int_ad_infec + df.unid_int_idoso_infec ) /
                ( df.unid_int_ad_cvc_dia + df.unid_int_idoso_cvc_dia ) )[0] *1000
kpi10_neo_uti = ( ( df.uti_neo_precoce_infec + df.uti_neo_tardio_infec ) / 
                    ( df.uti_neo_precoce_cvc_dia + df.uti_neo_tardio_cvc_dia ) )[0] *1000
kpi10_pedi_uti = ( df.uti_pedi_infec / df.uti_pedi_cvc_dia )[0] *1000
kpi10_ad_uti = ( ( df.uti_ad_infec + df.uti_idoso_infec ) / 
                    ( df.uti_ad_cvc_dia + df.uti_idoso_cvc_dia ) )[0] *1000
kpi10_unid_int = (
    (   df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec+
        df.unid_int_pedi_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec
    ) /                 
    (   df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia+
        df.unid_int_pedi_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia
    )
)[0] *1000
kpi10_uti = (
    (   df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.uti_pedi_infec+
        df.uti_ad_infec+
        df.uti_idoso_infec
    ) /                 
    (   df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.uti_pedi_cvc_dia+
        df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia
    )
)[0] *1000      
kpi10_neo = (
    (   df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec
    ) /                 
    (   df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia
    )
)[0] *1000
kpi10_pedi = (
    (   df.uti_pedi_infec+
        df.unid_int_pedi_infec
    ) /                 
    (   df.uti_pedi_cvc_dia+
        df.unid_int_pedi_cvc_dia
    )
)[0] *1000            
kpi10_ad = (
    (   df.uti_ad_infec+
        df.uti_idoso_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec
    ) /                 
    (   df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia
    )
)[0] *1000

kpi10 = (
    (   df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec+
        df.unid_int_pedi_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec+
        df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.uti_pedi_infec+
        df.uti_ad_infec+
        df.uti_idoso_infec
    ) /                 
    (   df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia+
        df.unid_int_pedi_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia+
        df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.uti_pedi_cvc_dia+
        df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia
    )
)[0] *1000

# 11. Densidade de incidência de infecção do trato urinário (ITU) associada a um
# cateter vesical de demora (CVD)
kpi11_neo = ( ( df.unid_int_neo_precoce_itu + df.unid_int_neo_tardio_itu ) / 
                ( df.unid_int_neo_precoce_cvd_dia + df.unid_int_neo_tardio_cvd_dia ) )[0] *1000
kpi11_pedi = ( df.unid_int_pedi_itu / df.unid_int_pedi_cvd_dia )[0] *1000
kpi11_ad = ( ( df.unid_int_ad_itu + df.unid_int_idoso_itu ) /
                ( df.unid_int_ad_cvd_dia + df.unid_int_idoso_cvd_dia ) )[0] *1000
kpi11_neo_uti = ( ( df.uti_neo_precoce_itu + df.uti_neo_tardio_itu ) / 
                    ( df.uti_neo_precoce_cvd_dia + df.uti_neo_tardio_cvd_dia ) )[0] *1000
kpi11_pedi_uti = ( df.uti_pedi_itu / df.uti_pedi_cvd_dia )[0] *1000
kpi11_ad_uti = ( ( df.uti_ad_itu + df.uti_idoso_itu ) / 
                    ( df.uti_ad_cvd_dia + df.uti_idoso_cvd_dia ) )[0] *1000
kpi11_unid_int = (
    (   df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu+
        df.unid_int_pedi_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu
    ) /                 
    (   df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia+
        df.unid_int_pedi_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia
    )
)[0] *1000
kpi11_uti = (
    (   df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.uti_pedi_itu+
        df.uti_ad_itu+
        df.uti_idoso_itu
    ) /                 
    (   df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.uti_pedi_cvd_dia+
        df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia
    )
)[0] *1000      
kpi11_neo = (
    (   df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu
    ) /                 
    (   df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia
    )
)[0] *1000
kpi11_pedi = (
    (   df.uti_pedi_itu+
        df.unid_int_pedi_itu
    ) /                 
    (   df.uti_pedi_cvd_dia+
        df.unid_int_pedi_cvd_dia
    )
)[0] *1000            
kpi11_ad = (
    (   df.uti_ad_itu+
        df.uti_idoso_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu
    ) /                 
    (   df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia
    )
)[0] *1000 
kpi11 = (
    (   df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu+
        df.unid_int_pedi_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu+
        df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.uti_pedi_itu+
        df.uti_ad_itu+
        df.uti_idoso_itu
    ) /                 
    (   df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia+
        df.unid_int_pedi_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia+
        df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.uti_pedi_cvd_dia+
        df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia
    )
)[0] *1000

# 12. Taxa de profilaxia de tromboembolismo venoso
kpi12_cli          = ( df.cli_profilaxia          / df.cli_total_pacientes )[0] *100
kpi12_cir_orto     = ( df.cir_orto_profilaxia     / df.cir_orto_total_pacientes )[0] *100
kpi12_cir_nao_orto = ( df.cir_nao_orto_profilaxia / df.cir_nao_orto_total_pacientes )[0] *100
kpi12 = ( ( df.cli_profilaxia+df.cir_orto_profilaxia+df.cir_nao_orto_profilaxia ) /
            ( df.cli_total_pacientes+df.cir_orto_total_pacientes+df.cir_nao_orto_total_pacientes )
        )[0] *100

# 13. Densidade de incidência de queda resultando em lesão em paciente
kpi13 = ( df.quedas_com_dano / 
            ( df.cli_neo_precoce_pacientes_dia+
            df.cli_neo_tardio_pacientes_dia+
            df.cli_pedi_pacientes_dia+
            df.cli_ad_pacientes_dia+
            df.cli_idoso_pacientes_dia+
            df.cir_neo_precoce_pacientes_dia+
            df.cir_neo_tardio_pacientes_dia+
            df.cir_pedi_pacientes_dia+
            df.cir_ad_pacientes_dia+
            df.cir_idoso_pacientes_dia )
            )[0] *1000

# 14. Evento sentinela
kpi14 = ( df.eventos_sentinela / 
            ( df.cli_neo_precoce_pacientes_dia+
            df.cli_neo_tardio_pacientes_dia+
            df.cli_pedi_pacientes_dia+
            df.cli_ad_pacientes_dia+
            df.cli_idoso_pacientes_dia+
            df.cir_neo_precoce_pacientes_dia+
            df.cir_neo_tardio_pacientes_dia+
            df.cir_pedi_pacientes_dia+
            df.cir_ad_pacientes_dia+
            df.cir_idoso_pacientes_dia )
        )[0] *1000

In [ ]:
kpis = [kpi1, 
kpi2_cli, kpi2_cir, kpi2,
kpi3,
kpi4_cli_neo_precoce, kpi4_cli_neo_tardio, kpi4_cli_pedi, kpi4_cli_adulto, kpi4_cli_idoso, kpi4_cli,
kpi4_cir_neo_precoce, kpi4_cir_neo_tardio, kpi4_cir_pedi, kpi4_cir_adulto, kpi4_cir_idoso, kpi4_cir,
kpi4_neo_precoce, kpi4_neo_tardio, kpi4_pedi, kpi4_ad, kpi4_idoso, kpi4,
kpi5_cli_pedi, kpi5_cli_adulto, kpi5_cli_idoso, kpi5_cli,
kpi5_cir_pedi, kpi5_cir_adulto, kpi5_cir_idoso, kpi5_cir,
kpi5_pedi, kpi5_ad, kpi5_idoso, kpi5,
kpi6,
kpi7_nvl2, kpi7_nvl3, kpi7,
kpi8,
kpi9,
kpi10_neo, kpi10_pedi, kpi10_ad, kpi10_neo_uti, kpi10_pedi_uti, kpi10_ad_uti, kpi10_unid_int, kpi10_uti, kpi10,
kpi11_neo, kpi11_pedi, kpi11_ad, kpi11_neo_uti, kpi11_pedi_uti, kpi11_ad_uti, kpi11_unid_int, kpi11_uti, kpi11,
kpi12_cli, kpi12_cir_orto, kpi12_cir_nao_orto, kpi12,
kpi13,
kpi14]